## OpenCL basics

#### This tutorial is build  and heavily inspired in

https://www.youtube.com/watch?v=RKyhHonQMbw


There are 2 subjects in an OpenCL program.

- **Host**: Piece of hardware that tells devices what to do to the devices. A host can give orders to several devices.


- **Device**: Piece of hardware that executes work for the host.


### Parts of a device

A device is a piece of hardware that is made of **compute units**, **Global Memory** and **Constant Memory**. Each compute unit has several processing elements.

- **Compute Unit (CU)**: A compute unit is made of processing elements (PE) and **local memory**.

    - **Local memory of a CU**: Memory shared across all processing elements of the CU. It is a very efficient way to share data acrross all PE elements of the CU. This data cannot be accessed by other compute units (that is why it is called local).

    - The following diagram shows a compute unit made of 6 processing elements. The compute unit has some local memory accesible to all processing elements. Moreover each processing element has some private memory.

```
    PE - private mem    PE - private mem
    PE - private mem    PE - private mem
    PE - private mem    PE - private mem
    [ ---------- Local Memory --------- ] 
``` 

    
- **Processing Element (PE) **: harware piece that executes instructions with a small **private memory**.


- **Global Memory**: Main memory of the device. This memory is shared with all processing elements. The host can access this memory. This could be useful for example to copy data from the host memory (usually RAM) to the device memory (for example GDDR5) or vice versa. 
    - This memory is persistent. If the host puts data in the Global memory and some computations are done, this data will still be there, unless the host explicitly frees the memory.
    

- **Constant Memory**: Is shared among all processing elements but it is **read-only memory**. It is a very efficient way to share data with all the PE of the device.


#### Summary of the types of memory:

- Device memory: **Global memory** and **Constant Memory**
- CU memory: **Local memory**
- PE memory: **private memory**


In [92]:
using OpenCL

const sum_kernel = "
   __kernel void sum(__global const float *a,
                     __global const float *b,
                     __global float *c)
    {
      int gid = get_global_id(0);
      c[gid] = a[gid] + b[gid];
    }
"
a = rand(Float32, 50_000)
b = rand(Float32, 50_000)

device, ctx, queue = cl.create_compute_context()

a_buff = cl.Buffer(Float32, ctx, (:r, :copy), hostbuf=a)
b_buff = cl.Buffer(Float32, ctx, (:r, :copy), hostbuf=b)
c_buff = cl.Buffer(Float32, ctx, :w, length(a))

p = cl.build!(cl.Program(ctx, source=sum_kernel))
k = cl.Kernel(p, "sum")

queue(k, size(a), nothing, a_buff, b_buff, c_buff)

r = cl.read(queue, c_buff);

In [71]:
using OpenCL

In [72]:
cl.devices()

3-element Array{OpenCL.cl.Device,1}:
 OpenCL.Device(Intel(R) Core(TM) i7-3720QM CPU @ 2.60GHz on Apple @0x00000000ffffffff)
 OpenCL.Device(HD Graphics 4000 on Apple @0x0000000001024400)                         
 OpenCL.Device(GeForce GT 650M on Apple @0x0000000001022700)                          

## OpenCL Host API

The main parts of the Host API are:

- **platform**: is an implementation of OpenCL. Platforms are drivers for the devices which expose devices to the host.

    - In this example we have a platform for the nvidia GPU and a platform for the intel HD4000 and a platform for the intel CPU. The platform discovers devices available to the host.


- **context**: Is a container with devices and memory.

    - You create a context for a specific platform
    - You cannot have multiple platforms in a context.
    - Most operations are related to a context. Explicitly or implicitly.


- **program**: Programs are just collections of kernels.
     - You must extract kernels from you program to call them
     - OpenCL applications have to load kernels.
     - Kernels have to be complied. They are OpenCL C source code.
     - Kernels can be loaded from a binary representation.
     - Programs are device specific.
    
    
#### Asynchronous  calls

The host manages devices asynchronously for best performance. The device management can be summarized as follows:

- Host issues commands to the device.
- Commands tell the device to do something.
- Devices take commands and do what is programmed.
- The host waits for commands to complete.
- Commands can be dependent on other commands.
- OpenCL commands are issued by **`clEnqueue`** calls.
    - A **`cl_event`** returned by **`clEnqueue*`** calls is used for dependencies.


#### Commands and command-queues


You can think as the Host to be the boss and devices are just people working for the boss. The command queue allows the boss to talk to each of the persons (devices). The boss can put work to the people.

OpenCL has commad-queues which allow the host to pass work to the devices. 

- A command-queue is a attached to a single device.
- There can be as many command-queues as you want.
- **`clEnqueue*`** commands have a command-queue parameter.

```
Host -->  [ command-queue  ]  --> Device
```

Let us assume we have several `cl_event` tasks.

```
Host -->  [ e1 e2 e3 e4 e5  ]  --> Device
```

The Device will pick one at a time and execute it. The host can find out that a command has been completed by the device.


In [73]:
ctx = cl.Context(cl.devices()[3])

OpenCL.Context(@0x00007fa130665e90 on GeForce GT 650M)

In [74]:
queue = cl.CmdQueue(ctx)

OpenCL.CmdQueue(@0x00007fa13050e640)

In [75]:
srand(123)
x = rand(Float32, 100000);
y = rand(Float32, 100000);
o = zeros(Float32,100000);

### Executing functions on a device


OpenCL executes kernel functions on the device.

yeah but... What is a kernel? kernels are functions written in C with some syntax sugar.

#### Kernel calls

Kernels calls contain 2 main parts: (argument_list, execution_parameters)

- Like most functions they have a function argument list.
- They have **external execution parameters** that control parallelism.

### Host and Device rols

The Host coordinates the execution of kernel calls. Nevertheless kernels are executed on the device. 

The Host has to 

- Provide arguments to the kernel.
- Provide execution parameters (paralelism control) to launch the kernel.


### What the host does

Paralelism is created by invoking the same kernel function many times. The argument list of a kernel is  identical for all invocations. Notice that invocation is different than kernel call.

You run the kernel once and then the kernel is invoked many times with the same argument list for all invocations.

The strategy for opencl is to invoke the same function over and over. The amount of times is invoked is given by the execution parameters of the kernel call.

The host needs to set extra execution parameters prior to launch a kernel. 

### Index Space: NDRange

The paradigm of parallel computation in OpenCL is designed around having the same operations on different data. We have a single kernel that is runned may times on different data slices.

How do kernels know in what data they should be working? If the argument list is identicall for all invocations of the kernel this seems imposible to do.

A kernel knows what data has to work on because ** execution parameters provide an index space** and **each function invocation can access its index**. 
The index space is n-Dimensional.



#### Loop example

Let us consider we want to compute `func(a,b)` 10 times. A standard way could be using a for loop:

```C
for (size_t global_id=0; global_id < global_work_size, ++global_id)
{
    func(a,b);
}
```

In opencl there is a particular terminology to refer to the different elements "in a for loop". Notice that we used

- **`global_id`** to the index traking the iteration.

- **`global_work_size`** to the total number of iterations.

If we have a situation where there is an offset to the for loop we call the offset **"global work offset"**.

```C
for (size_t global_id = global_work_offset; global_id < global_work_size +  global_work_offset, ++global_id)
{
    func(a,b);
}
```

- **`work dimension`** corresponds to the number of for loops that control kernel invocations.

In the wollowing example the `work dimension` would be 3.
```C
for (size_t i =0, i < size_i; ++i)
    for (size_t j =0, j < size_j; ++j)
        for (size_t k=0, k < size_k; ++k)
            func(a,b);
```




#### NDrange as index Space

- Do not think of for loops since they are inherently sequential.
- Think of a set of indicies where each element in the set is a tuple of dimension ND.
- Each invocation of a kernel pulls a random index form the set of indices.
- The index Space is populated before kernel execution.
- An invoked kernel picks an index form the set and runs.
- The kernel call stops when the index Space is empty.

#### Definitions NDRange

- **Work-item** is an invocation of a kernel for a particular index.


- **Global ID**: Globally unique id for a work-item (from the index space).


- **Global work size**: Number of work-items (per dimension).


- ** Work dimension**: Dimension of the index space.


In [77]:
sum_cl = "
#pragma OPENCL EXTENSION cl_khr_byte_addressable_store : enable
__kernel void cl_sum(__global float32 *x,
                     __global float32 *y, 
                     __global float32 *o)
{
    int gid = get_global_id(0);
    o[gid] = x[gid] + y[gid];
}";

### Execution model 

When we want to invoke a kernel many times we build an index pace. From the index space a kernel picks an index invokes itself with that index and removes the index from the index space. 

Remember that all invocations have the same argument list.


- **Work-item** is an invocation of a kernel for a particular index.

- **Global ID**: Globally unique id for a work-item (from the index space).

- **Global work size**: Number of work-items (per dimension).

- ** Work dimension**: Dimension of the index space.



### Device model

We have talked about devices beeing a collection of compute units with global and constant memory. Where each compute unit is a collection of procesing elements sharing local memory.

### Execution model and Device model

Since processing elements in the device run instructions we should run our code on them. Which means that **work-items should work on processing elements**.

Since work-items are sinmply invocations of the kernel we want to assign multiple work items to each processing element. Notice that we want this since we have to think about how to handle the case of having a bigger global work size than the number of processing elements.

#### Work-groups and work-items

kernel invocations are done by processing elements. Processing elements are placed into compute units each of which contains local memory. Since kernel invocations probably need some data we want to use the **local memory** in every compute unit insted of the global memory. To do so we will partition the global work into smaller pieces.

Each partition of the global work is called a work-group. Notice that:

- Compute Unit (CU) is a collection of Processing elements (PE).
- A work-group (WG) is a collection of Work-items (WI).

Therefore there is a natural correspondence between copute units and work-groups.

We will execute work-groups in compute units.
- Compute unit local memory is shared by the work-group. That means that all work-items in the work group share local memory.
- Work-items in the work-group are mapped to processing elements in the Compute unit.

Since the number of processing elements in a compute unit is "device specific" we want the work-group size to match the number of processing elements in a compute unit.


#### Work item world

A work-item can access different types of memory:

- Private memory from the work group
- Constant memory
- Global memory

A work-item can know:

- The work-group id
- The size fo work-groups
- The global id
- The global work size

#### Work-group size

The maximum work-group size even though it its defined by software is a device characteristic. You can query the device to determine this value.

In [95]:
queue = cl.CmdQueue(ctx)

OpenCL.CmdQueue(@0x00007fa1301af2f0)

In [96]:
x_buff = cl.Buffer(Float32, ctx, (:r, :copy), hostbuf=x)
y_buff = cl.Buffer(Float32, ctx, (:r, :copy), hostbuf=y)
o_buff = cl.Buffer(Float32, ctx, (:r, :copy), hostbuf=o)

Buffer{Float32}(@0x00007fa130689170)

In [97]:
prg = cl.Program(ctx, source=sum_cl) |> cl.build!

OpenCL.Program(@0x00007fa13057d660)

In [98]:
k = cl.Kernel(prg, "cl_sum")

OpenCL.Kernel("cl_sum" nargs=3)

In [99]:
queue(k, length(x), nothing, x_buff, y_buff, o_buff)
#cl.copy!(queue, out, o_buff)

OpenCL.Event(@0x00007fa130651fa0)

In [100]:
r = cl.read(queue, c_buff);

In [102]:
if isapprox(norm(r - (a+b)), zero(Float32))
    info("Success!")
else
    error("Norm should be 0.0f")
end


INFO: Success!


## n-dimensional work-groups


Work groups can have multiple dimensions.
This can be interpreted as

- Geometrically
- Pulling n-dimensional tuples from  a set.

The device maximum work-group size is a single integer. For example, 32. Nevertheless Work-groups can be n-dimensional. For example the maximum work-group size could be 32 but work-groups could be launched with (8,2,1) dimensions. 

It is important to know that for the device the work-group size is 1 dimensional. A multidimensional work group size is simply an abstraction for the programmer.

For example we could have some code where the work-group size is  `(w1,w2,...,wk)`. As long as `w1*w2*...*wk<=max` where `max` is the maximum work-group size the code will run fine. If `w1*w2*...*wk>max` then the host API will return an error.


#### Vector example |work-groups| <= |CU|

Assume `global_work_size=32` and `work_group_size=8`. Given a vector of length 32 OpenCL would automatically

- Make 4  work-groups containing 8 work-items. 
- Each work group would given to a compute unit.
- Each work group will give a work item to a processing element automatically.


#### Vector example |work-groups| > |CU|

Assume `global_work_size=32` and `work_group_size=8`. Given a vector of length 32 OpenCL and assuming for example we have a single compute unit, openCL will invoke one work group at a time until the 4 work-groups have been executed.



#### Kernel calls

Host must provide execution dimensions to the device. This will create an index space.

Remember that Global memory is persistant between kernel invocations but Constant, Local and Private memory is just scratch space as is reset per kernel call.
